# Generate subjectwise results

This notebook contains code that will generate subject-wise results to compare ground truth predictions and C3D generated dose volumes. We are still exploring how to measure the 'quality' of the reconstruction, and some of the results here can hopefully clarify how to measure this more accurately. 

In [1]:
# Setup library paths
import os
import numpy as np
import SimpleITK as sitk

from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pdf
%matplotlib inline
plt.rcParams['figure.figsize'] = [30, 15]

In [4]:
root_path = "/Users/amithkamath/repo/deepdosesens"
base_gt_path = os.path.join(root_path, "data", "processed-dldp")
base_pred_path = os.path.join(root_path, "data", "output-dldp", "output-dldp-6", "Prediction")
results_path = os.path.join(root_path, "results")

In [5]:
# Loop through the selected files and generate a pdf: with representative slices 
pp = pdf.PdfPages(os.path.join(results_path, "1_deep_plan_results.pdf"))

subjects = ["DLDP_" + str(x).zfill(3) for x in range(81, 101)]

for subject in subjects:
        # Report the current state
        print('Analyzing subject ' + subject)

        # Read GT
        gt_path = os.path.join(base_gt_path, subject, "Dose.nii.gz")
        gt_image = sitk.ReadImage(gt_path)
        gt_array = sitk.GetArrayFromImage(gt_image)

        # Read Pred
        pred_path = os.path.join(base_pred_path, subject, "Dose.nii.gz")
        pred_image = sitk.ReadImage(pred_path)
        pred_array = sitk.GetArrayFromImage(pred_image)
        
        diff_array = pred_array - gt_array

        # Get the samples from the image
        selected_idx = np.linspace(0.0, 1.0, num=30)

        for index in selected_idx:
            slice_num = int((gt_array.shape[0] - 1) * index)
            fig, axes = plt.subplots(1, 3)
            im_gt = axes[0].imshow(gt_array[slice_num, :, :], cmap='coolwarm', vmin=0, vmax=70)
            #axes[0].title.set_text("GT")
            im_pred = axes[1].imshow(pred_array[slice_num, :, :], cmap='coolwarm', vmin=0, vmax=70)
            #axes[1].title.set_text("Pred")
            diff = axes[2].imshow(diff_array[slice_num, :, :], cmap='bwr', vmin=-15, vmax=15)

            p0 = axes[0].get_position().get_points().flatten()
            p1 = axes[1].get_position().get_points().flatten()
            p2 = axes[2].get_position().get_points().flatten()

            ax_cbar = fig.add_axes([p0[0], 0.2, p1[2]-p0[0], 0.05])
            plt.colorbar(im_gt, cax=ax_cbar, orientation='horizontal')
            
            ax_cbar1 = fig.add_axes([p2[0], 0.2, p2[2]-p2[0], 0.05])
            plt.colorbar(diff, cax=ax_cbar1, orientation='horizontal')

            fig.suptitle("Subject: " + str(subject) + " slice: " + str(slice_num))
            pp.savefig(fig)
            plt.close()

pp.close()

Analyzing subject DLDP_081
Analyzing subject DLDP_082
Analyzing subject DLDP_083
Analyzing subject DLDP_084
Analyzing subject DLDP_085
Analyzing subject DLDP_086
Analyzing subject DLDP_087
Analyzing subject DLDP_088
Analyzing subject DLDP_089
Analyzing subject DLDP_090
Analyzing subject DLDP_091
Analyzing subject DLDP_092
Analyzing subject DLDP_093
Analyzing subject DLDP_094
Analyzing subject DLDP_095
Analyzing subject DLDP_096
Analyzing subject DLDP_097
Analyzing subject DLDP_098
Analyzing subject DLDP_099
Analyzing subject DLDP_100


In [ ]:
# Loop through the selected files and generate a pdf with cumulative histogram of differences
pp = pdf.PdfPages(os.path.join(results_path, '1_deep_plan_plain_histogram_without_center.pdf'))

for subject in subjects:
        # Report the current state
        print('Analyzing subject ' + subject)

        # Read GT
        gt_path = os.path.join(base_gt_path, subject, "Dose.nii.gz")
        gt_image = sitk.ReadImage(gt_path)
        gt_array = sitk.GetArrayFromImage(gt_image)

        # Read Pred
        pred_path = os.path.join(base_pred_path, subject, "Dose.nii.gz")
        pred_image = sitk.ReadImage(pred_path)
        pred_array = sitk.GetArrayFromImage(pred_image)
        
        diff_array = pred_array - gt_array
        n_elems = np.prod(diff_array.shape)
        res, bin_lims = np.histogram(diff_array, bins=100, range=(-15, 15))
        

        # Make central peak 0 to highlight the errors only
        res[res == np.max(res)] = 0
        plt.bar(bin_lims[:-1], res / n_elems, width=0.3, color="blue")
        ax = plt.gca()
        ax.set_ylim([0, 0.12])
        plt.title("Subject: " + str(subject))
        plt.yticks(np.arange(0, 0.1, step=0.01))
        plt.ylabel("Percentage of voxels with specified difference in Grays")
        plt.xlabel("Difference in grays: Pred - GT; -ve = pred understimate, +ve = pred overestimate")
        plt.grid()
        pp.savefig()
        plt.close()

pp.close()

Analyzing subject DLDP_081
Analyzing subject DLDP_082
Analyzing subject DLDP_083
Analyzing subject DLDP_084
Analyzing subject DLDP_085
Analyzing subject DLDP_086
Analyzing subject DLDP_087
Analyzing subject DLDP_088
Analyzing subject DLDP_089
Analyzing subject DLDP_090
Analyzing subject DLDP_091
Analyzing subject DLDP_092
Analyzing subject DLDP_093
Analyzing subject DLDP_094
Analyzing subject DLDP_095
Analyzing subject DLDP_096
Analyzing subject DLDP_097
Analyzing subject DLDP_098
Analyzing subject DLDP_099
Analyzing subject DLDP_100


In [ ]:
for subject in subjects:
        # Report the current state
        print('Analyzing subject ' + subject)

        # Read GT
        gt_path = os.path.join(base_gt_path, subject, "Dose.nii.gz")
        gt_image = sitk.ReadImage(gt_path)
        gt_array = sitk.GetArrayFromImage(gt_image)

        target_volume_path = os.path.join(base_gt_path, subject, "Target.nii.gz")
        target_image = sitk.ReadImage(target_volume_path)
        target_array = sitk.GetArrayFromImage(target_image)

        brain_volume_path = os.path.join(base_gt_path, subject, "Brain.nii.gz")
        brain_image = sitk.ReadImage(brain_volume_path)
        brain_array = sitk.GetArrayFromImage(brain_image)

        relative_target_volume = np.sum(target_array) / np.sum(brain_array)

        print("For subject: ", subject, " relative target size is: ", relative_target_volume)        

For subject:  DLDP_081  relative target size is:  0.07551555654108777
For subject:  DLDP_082  relative target size is:  0.012293136850622827
For subject:  DLDP_083  relative target size is:  0.13434883720930232
For subject:  DLDP_084  relative target size is:  0.03229032123661514
For subject:  DLDP_085  relative target size is:  0.07755061334397127
For subject:  DLDP_086  relative target size is:  0.2293007172991059
For subject:  DLDP_087  relative target size is:  0.06004682702714642
For subject:  DLDP_088  relative target size is:  0.10329059197398566
For subject:  DLDP_089  relative target size is:  0.026332849953083683
For subject:  DLDP_090  relative target size is:  0.027187992791169183
For subject:  DLDP_091  relative target size is:  0.1004198167288777
For subject:  DLDP_092  relative target size is:  0.08054732041049031
For subject:  DLDP_093  relative target size is:  0.07209882504426203
For subject:  DLDP_094  relative target size is:  0.08210575632258471
For subject:  DLDP_